# ReAct Agent with Query Engine Tools

## Build Query Engine Tools

In [1]:
from llama_index import SimpleDirectoryReader, VectorStoreIndex, StorageContext, load_index_from_storage

from llama_index.tools import QueryEngineTool, ToolMetadata

In [2]:
try:
    storage_context = StorageContext.from_defaults(persist_dir='./storage/lyft')
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(persist_dir='./storage/uber')
    uber_index = load_index_from_storage(storage_context)
    
    index_loaded = True
except:
    index_loaded = False

In [3]:
if not index_loaded:
    # load data 
    lyft_docs = SimpleDirectoryReader(input_files=["../data/10k/lyft_2021.pdf"]).load_data()
    uber_docs = SimpleDirectoryReader(input_files=["../data/10k/uber_2021.pdf"]).load_data()
    
    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)
    
    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

In [4]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [5]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine, 
        metadata=ToolMetadata(
            name='lyft_10k', 
            description="Provides information about Lyft financials for year 2021. "
            "Use a detailed plain text question as input to the tool."
        )
    ),
    QueryEngineTool(
        query_engine=uber_engine, 
        metadata=ToolMetadata(
            name='uber_10k', 
            description="Provides information about Uber financials for year 2021. "
            "Use a detailed plain text question as input to the tool."
        )
    ),
]


## Setup ReAct Agent

In [6]:
from llama_index.agent import ReActAgent

In [7]:
agent = ReActAgent.from_tools(
    query_engine_tools,
)

In [8]:
agent.chat("What was Lyft's revenue growth in 2021?")

Thought: I can use the lyft_10k tool to find information about Lyft's revenue growth in 2021.
Action: lyft_10k
Action Input: {"input": "What was Lyft's revenue growth in 2021?"}
Thought: I have enough information to answer the question without using any more tools.
Answer: Lyft's revenue growth in 2021 was 36%.


Response(response="Lyft's revenue growth in 2021 was 36%.", source_nodes=[], metadata=None)

## Let's Try It Out!

In [8]:
agent.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.




KeyboardInterrupt

